# Recap

In [1]:
%%writefile main.kv

<Widget>:
    canvas:
        Color:
            rgba: .5,.5,.5,.5
        Line:
            rectangle: self.x, self.y, self.width, self.height
            width: 2

Overwriting main.kv


In [2]:
%%writefile main.py

from kivy.app import App
from selectiontool import SelectionTool


class MainApp(App):
    title = 'Image Selection Tool'

    def build(self):
        return SelectionTool()


if __name__ == '__main__':
    MainApp().run()

Overwriting main.py


In [3]:
%%writefile imagepane.py

from kivy.uix.image import Image
from kivy.lang import Builder
from kivy.app import App

Builder.load_string("""
<ImagePane>:
    allow_stretch: True
""")

class ImagePane(Image):
    pass


Overwriting imagepane.py


In [4]:
%%writefile selectiontool.kv

<SelectionTool>:
    orientation: "vertical"

    book_selector: _book_selector
    page_selector: _page_selector
    image_pane: _image_pane
    word_list: _word_list

    book_id: self.book_selector.text
    page: self.page_selector.text

    BoxLayout:
        orientation: 'horizontal'

        Widget: # Spacer
            size_hint_x: 4

        Spinner:
            id: _book_selector

        Spinner:
            id: _page_selector

    Button:
        text: "Delete Last Rectangle"
        on_press: root.image_pane.delete_last_rectangle()

    BoxLayout:
        orientation: 'horizontal'
        size_hint_y: 16
        BoxLayout:
            id: _word_list
            size_hint_x: 0.2
            orientation: 'vertical'

        ImagePane:
            id: _image_pane
            size_hint_x: 0.8
            source: ''

Overwriting selectiontool.kv


# Slightly modified selectiontool.py

In [5]:
%%writefile selectiontool.py

import os
import glob
import json

from kivy.uix.boxlayout import BoxLayout
from kivy.uix.label import Label
from kivy.properties import StringProperty
from kivy.lang import Builder

from imagepane import ImagePane
from selectionbox import SelectionBox  # added for later


Builder.load_file('selectiontool.kv')


class SelectionTool(BoxLayout):
    library_directory = 'Example_Data'

    book_id = StringProperty()
    page = StringProperty()

    def __init__(self):
        super(SelectionTool, self).__init__()
        book_pattern = os.path.join(self.library_directory, '[0-9]' * 4)
        self.book_selector.values = [os.path.basename(s) for s in glob.glob(book_pattern)]
        self.book_selector.text = self.book_selector.values[0] if self.book_selector.values else 'No Books'

    def on_book_id(self, inst=None, value=None):
        image_pattern = os.path.join(self.library_directory, self.book_id, '*.jpg')
        self.page_selector.values = [os.path.basename(s)[:-4] for s in glob.glob(image_pattern)]
        self.page_selector.text = self.page_selector.values[0] if self.page_selector.values else 'No Images'

        self.word_list.clear_widgets()
        with open(os.path.join(self.library_directory, self.book_id, 'word_list.txt')) as fp:
            for word in sorted(fp.readlines()):
                self.word_list.add_widget(Label(text=word.strip()))
        
        self.on_page()

    def on_page(self, *_):
        page_filename = self.page + '.jpg'
        self.image_pane.source = os.path.join(self.library_directory, self.book_id, page_filename)
        

Overwriting selectiontool.py


# Rectangles!

Now we're going to add code to draw rectanges on the image when we drag our finger / mouse. 

We will make a subclass of `Widget` to do this.

In [6]:
%%writefile selectionbox.kv

<SelectionBox>:
    label: _label
    color: (1, 0, 0, 1)
    image_pane: None
    canvas:
        Color:
            rgba: root.color
        Line:
            width: 2.
            rectangle: (self.x, self.y, self.width, self.height)
    TextInput:
        id: _label
        multiline: False
        size_hint: 1, None
        height: 50
        center: root.center
        on_text: root.image_pane.store_rectangles() if root.image_pane else None


Overwriting selectionbox.kv


The `canvas` instruction allows you to draw on the screen. Here we draw an unfilled rectangle. We are also going to keep a pointer to the `ImagePane` that the rectangle is being drawn in.

The `TextInput` widget is, as you might guess, a box that we can type in. Here the line
```        
on_text: root.image_pane.store_rectangles() if root.image_pane else None
```
calls the image_pane's `store_rectangles()` method.



In [7]:
%%writefile selectionbox.py

from kivy.uix.widget import Widget
from kivy.lang import Builder

Builder.load_file('selectionbox.kv')


class SelectionBox(Widget):

    def __init__(self, image_pane, text='', pos=None, size=None, unit_pos=None, unit_size=None):
        super(SelectionBox, self).__init__(pos=pos, size=size)
        self.label.text = text
        self.image_pane = image_pane
        self.unit_pos = unit_pos
        self.unit_size = unit_size
        
    def to_dict(self):
        return {'text': self.label.text, 'pos': self.pos, 'size': self.size,
                'unit_pos': self.unit_pos, 'unit_size': self.unit_size}

    def compute_unit_coordinates(self):
        image_pos = [self.image_pane.pos[n] + (self.image_pane.size[n] - self.image_pane.norm_image_size[n]) / 2
                     for n in (0, 1)]
        self.unit_pos = tuple([(self.pos[n] - image_pos[n]) * 1.0 / self.image_pane.norm_image_size[n] for n in [0, 1]])
        self.unit_size = tuple([self.size[n] * 1.0 / self.image_pane.norm_image_size[n] for n in [0, 1]])

    def compute_screen_coordinates(self, *_):
        image_pos = [self.image_pane.pos[n] + (self.image_pane.size[n] - self.image_pane.norm_image_size[n]) / 2
                     for n in (0, 1)]
        self.pos = [self.unit_pos[n] * self.image_pane.norm_image_size[n] + image_pos[n] for n in [0, 1]]
        self.size = [self.unit_size[n] * self.image_pane.norm_image_size[n] for n in [0, 1]]


Overwriting selectionbox.py


We have a few methods here, basically coordinate conversion.

We want our `ImagePane` class to created boxes when you drag on the image. When you interact the screen Kivy generates touch events, and will call the corresponding `on_touch` methods.

In [8]:
%%writefile imagepane.py

from kivy.uix.image import Image
from kivy.lang import Builder
from kivy.app import App

from selectionbox import SelectionBox

Builder.load_string("""
<ImagePane>:
    allow_stretch: True
""")

class ImagePane(Image):

    drawing_rectangle = None # rectangle we are in the middle of drawing
    rectangles = []

    def on_touch_move(self, touch):
        if self.collide_point(*touch.pos): # did you touch the ImagePane?
            pos = [min(touch.pos[n], touch.opos[n]) for n in [0, 1]]
            size = [abs(touch.pos[n] - touch.opos[n]) for n in [0, 1]]
            if self.drawing_rectangle is None:
                self.drawing_rectangle = SelectionBox(pos=pos, size=size, image_pane=self)
                self.add_new_rectangle(self.drawing_rectangle)
            else:
                self.drawing_rectangle.pos = pos
                self.drawing_rectangle.size = size

    def on_touch_up(self, touch):
        if self.drawing_rectangle:
            self.drawing_rectangle.compute_unit_coordinates()
            self.drawing_rectangle = None
            self.store_rectangles()

    def add_new_rectangle(self, rect):
        self.add_widget(rect) # this adds the rect Widget to the ImagePane instance programmatically
        self.rectangles.append(rect)
        
    def delete_last_rectangle(self):
        if self.rectangles:
            bad_rectangle = self.rectangles.pop()
            self.remove_widget(bad_rectangle) # this removes the bad_rectangle widget from the ImagePane
            self.store_rectangles()

    def store_rectangles(self):
        pass  # for now


Overwriting imagepane.py


Note that when there's a touch event, the event goes to all widgets. So the `ImagePane` has to check to see if you're touching it (rather than selecting a spinner, for example).

In [9]:
#!python main.py

<img src="Images/first_rectangle.png"/>

The delete button doesn't crash the app anymore!

However, when we change pages the boxes are still there. Note that in `on_page()` we change the `ImagePane` `source`. This loads the new image. We don't create a new `ImagePane` object. Thus the `SelectionBox` widgets that we've added to the `ImagePane` are still there.

In [10]:
%%writefile -a imagepane.py

    def clear_rectangles(self):
        self.rectangles = []
        self.clear_widgets()


Appending to imagepane.py


In [11]:
%%writefile -a selectiontool.py

    def on_page(self, *_):
        page_filename = self.page + '.jpg'
        self.image_pane.source = os.path.join(self.library_directory, self.book_id, page_filename)
        self.image_pane.clear_rectangles() # added

Appending to selectiontool.py


In [12]:
#!python main.py

They're gone now... For good. That's not exactly what we want. Let's keep track of all the rectangles for all the books and pages, then in `on_page()` we can add the rectangles for that page.

# Keep track of the rectangles

Let's store all the rectangles in a dict keyed by `book_id` and `page`.

In [13]:
%%writefile -a selectiontool.py

    def __init__(self):
        super(SelectionTool, self).__init__()
        self.all_rectangles = {} # this is new

        book_pattern = os.path.join(self.library_directory, '[0-9]' * 4)
        self.book_selector.values = [os.path.basename(s) for s in glob.glob(book_pattern)]
        self.book_selector.text = self.book_selector.values[0] if self.book_selector.values else 'No Books'

        self.on_book_id()
    
    def on_page(self, *_):
        page_filename = self.page + '.jpg'
        self.image_pane.source = os.path.join(self.library_directory, self.book_id, page_filename)
        self.image_pane.clear_rectangles()
        try:
            for rect in self.all_rectangles[self.book_id][self.page]: # add this try/except block
                self.image_pane.add_new_rectangle(rect)
        except KeyError:
            pass

    def store_rectangles(self, rect_list):
        if self.book_id not in self.all_rectangles:
            self.all_rectangles[self.book_id] = {}
        self.all_rectangles[self.book_id].update({self.page: [r for r in rect_list]})

Appending to selectiontool.py


In [14]:
%%writefile -a imagepane.py

    def store_rectangles(self):
        App.get_running_app().root.store_rectangles(self.rectangles)


Appending to imagepane.py


In [15]:
#!python main.py

# Load and save rectangles

Finally let's keep the rectangles on disk.

In [16]:
%%writefile -a selectiontool.py
    
    def __init__(self):
        super(SelectionTool, self).__init__()
        self.all_rectangles = {}
        self.rectangles_filename = 'rectangles.json'  # added
        self.load_rectangles()  # added
        
        book_pattern = os.path.join(self.library_directory, '[0-9]' * 4)
        self.book_selector.values = [os.path.basename(s) for s in glob.glob(book_pattern)]
        self.book_selector.text = self.book_selector.values[0] if self.book_selector.values else 'No Books'

        self.on_book_id()

    def store_rectangles(self, rect_list):
        if self.book_id not in self.all_rectangles:
            self.all_rectangles[self.book_id] = {}
        self.all_rectangles[self.book_id].update({self.page: [r for r in rect_list]})
        self.save_rectangles()  # added

    def load_rectangles(self):
        self.all_rectangles = {}
        try:
            with open(self.rectangles_filename) as fd:
                all_rectangles_dict = json.load(fd)
                for book_id, book_rectangles in all_rectangles_dict.items():
                    self.all_rectangles[book_id] = {}
                    for page, rectangles in book_rectangles.items():
                        self.all_rectangles[book_id][page] = \
                            [SelectionBox(image_pane=self.image_pane, **rect) for rect in rectangles]
        except IOError:
            print("Can't find rectangles file!")
        
    def save_rectangles(self):
        rectangle_dict = {}
        for book_id, book_rectangles in self.all_rectangles.items():
            for page, rectangles in book_rectangles.items():
                page_dict = {page: [rect.to_dict() for rect in rectangles]}
                if rectangles:
                    rectangle_dict[book_id] = rectangle_dict.get(book_id, {})
                    rectangle_dict[book_id].update(page_dict)
        
        with open(self.rectangles_filename, 'w') as fd:
            json.dump(rectangle_dict, fd, sort_keys=True, indent=4, separators=(',', ': '))


Appending to selectiontool.py


In [17]:
#!python main.py

In [18]:
!cat rectangles.json

{
    "0001": {
        "01": [
            {
                "pos": [
                    488.0,
                    895.9999999999999
                ],
                "size": [
                    170.0,
                    148.0000000000001
                ],
                "text": "sun",
                "unit_pos": [
                    0.05750000000000003,
                    0.8399999999999999
                ],
                "unit_size": [
                    0.159375,
                    0.1387500000000001
                ]
            }
        ]
    }
}

# Exercise

Add a button that toggles your name appearing in the middle of the image.